In [ ]:
import pandas as pd
import json
from urllib.request import urlopen, quote
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.tsa.stattools import acf, pacf
from hmmlearn.hmm import GaussianHMM
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.stats import sem, t
from scipy import mean

In [ ]:
def get_sequence(province, filepath):
    url = 'https://lab.isaaclin.cn/nCoV/api/area?latest=0&province='+quote(province)
    req = urlopen(url)
    res = req.read().decode() 
    temp = json.loads(res)
    with open(filepath, 'w') as outfile:
        json.dump(temp, outfile)
    return temp

In [ ]:
def fetch_city(datapath, city):
    with open(datapath) as json_file:
        data = json.load(json_file)
        
    citylist = []
    for i in data["results"]:
        try:
            for cc in i['cities']:
                if cc['cityName'] == city:
                    citylist.append({**{'updateTime': datetime.fromtimestamp(i['updateTime'] // 1000).strftime("%m/%d/%Y")}, **cc})
        except Exception as e:
            print (e)
    data = pd.DataFrame.from_dict(citylist, orient = 'columns')
    data.drop_duplicates(subset=['updateTime'], keep='first', inplace=True)
    data = data.iloc[::-1].reset_index().drop(columns=['index'])
    return data

In [ ]:
def simulation(data, trace, totalpopulation, qu):
    simulated = []
    real = list(data['infectious'])
    n = len(real)
    i = 0
    while i<200:
        if i<n:
            theta = trace['theta_'+str(i)]
            simulated.append(theta[1]*totalpopulation)
            i += 1
        else:

            ka = trace['ka']
            Lambda = trace['Lambda1']
            gamma = trace['gamma']
            beta = trace['beta']
            solve_theta = ka*np.array([theta[0]-qu[i]*beta*theta[0]*theta[1], 
                                           theta[1]+qu[i]*beta*theta[0]*theta[1]-gamma*theta[1], 
                                            theta[2] + gamma*theta[1]])
    
            theta = np.random.dirichlet(alpha = solve_theta)
            if theta[1]<=0:
                break
            pred_infect = np.random.beta(a = Lambda*theta[1], b = Lambda*(1-theta[1]))
            simulated.append(pred_infect*totalpopulation)

            i += 1
    return simulated, real

In [ ]:
def draw_plot(origin,pred,lower,upper,title):
    x1 = origin
    x2 = pred
    plt.plot(x1, label = "True_value")
    plt.plot(x2, 'c--', label = "Predicted_value")
    plt.plot(lower, 'r--', label = '95%_CI_lower')
    plt.plot(upper, 'r--', label = '95%_CI_upper')
    plt.xlabel('Time (days)')
    plt.ylabel('Infections')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig('simulation.png')
    plt.show()
    return

In [ ]:
def calculate_quar(ntotal):
    coeff = np.zeros(ntotal)
    for i in range(ntotal):
        coeff[i] = max(0.2, 1-i*0.032)
    return coeff

In [ ]:
def CI(confidence, data):
    n = len(data)
    m = mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    lower = m - h
    upper = m + h
    return lower, upper

In [ ]:
if __name__ == "__main__":
    
    wholedata = get_sequence('湖北省','hubeidata.txt')
    data = fetch_city('hubeidata.txt','武汉')
    totalpopulation = 11080000
    mu, delta = calculate_rate(data, totalpopulation)        
    basic_model = pm.Model()
    n = len(data)
    infectious = np.array(data['infectious'])
    recovered = np.array(data['curedCount'])
    dead = np.array(data['deadCount'])
    S = totalpopulation - infectious[0] - recovered[0] - dead[0]
    I = infectious[0]
    R = recovered[0] + dead[0]
    qu = calculate_quar(300)
    
    
    
    with basic_model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0, upper=1.0)
        BoundedNormal2 = pm.Bound(pm.Normal, lower=0.0)
        theta = []
        #beta = pm.Beta('beta', 0.5, 0.5)
        #gamma = pm.Beta('gamma', 0.5, 0.5)
        #beta = BoundedNormal('beta', mu = 0.05, sigma = 0.1)
        r0 = BoundedNormal2('R_0', mu = (2+3.5)/2, sigma = 1)
        gamma = BoundedNormal('gamma', mu = 1/(17+14), sigma = 0.1)
        beta = pm.Deterministic('beta', r0*gamma)
        ka = pm.Gamma('ka', 2, 0.0001)
        Lambda1 = pm.Gamma('Lambda1', 2, 0.0001)
        Lambda2 = pm.Gamma('Lambda2', 2, 0.0001)
        
        solve_theta = pm.Deterministic('solve_theta_'+str(0), ka*pm.math.stack([S, I, R]))
        theta.append(pm.Dirichlet('theta_0', a = solve_theta, shape = (3)))
        for i in range (1,n):
            states = theta[i-1]
            solve_theta = pm.Deterministic('solve_theta_'+str(i), ka*pm.math.stack([states[0]-qu[i]*beta*states[0]*states[1], 
                                           states[1]+qu[i]*beta*states[0]*states[1]-gamma*states[1], 
                                            states[2] + gamma*states[1]]))
            theta.append(pm.Dirichlet('theta_'+str(i), a = solve_theta, shape = (3)))
            real_infect = pm.Beta('real_infect_'+str(i), Lambda1*theta[i][1], Lambda1*(1-theta[i][1]), observed = infectious[i]/totalpopulation)
            real_recover = pm.Beta('real_revocer_'+str(i), Lambda2*theta[i][2], Lambda2*(1-theta[i][2]), observed = recovered[i]/totalpopulation)
            
        step = pm.Metropolis()
        trace = pm.sample(2000, cores=16, chains = 1, init='auto', step = step)
        
    with open('dirichlet_model_with_quarantine.pkl', 'wb') as buff:
        pickle.dump({'model': basic_model, 'trace': trace}, buff)

In [ ]:
    '''
    with open('dirichlet_model_without_quarantine.pkl', 'rb') as buff:
        filedata = pickle.load(buff)
    basic_model, trace = filedata['model'], filedata['trace']
    #pm.plot_posterior(trace, varnames = ['beta', 'gamma'])
    
    
    #results, real = simulation(data, trace[1000], totalpopulation, qu)           
    #draw_plot(real, results, 'simulation')
    
    nsample = 5000
    total_results = np.zeros((nsample, 200))
    burn = 100
    for i in range(55,300):
        qu[i] = min(0.5,qu[i]+0.015*(i-55))
    
    for i in range(nsample):
        if i%500==0:
            print (i)
        k = np.random.randint(burn,2000)
        results, real = simulation(data, trace[k], totalpopulation, qu)
        for j in range(len(results)):
            total_results[i,j] = results[j]
    
    average = np.mean(total_results, axis = 0)
    lower = []
    upper = []
    for i in range(200):
        l,u = CI(0.95, total_results[:,i])
        lower.append(l)
        upper.append(u)
        
    draw_plot(real, average,lower,upper, 'simulation')
    '''       